In [1]:
import pysam
import collections
from math import floor
from pprint import pprint
import sys

In [42]:
#funcion para pasar las posiciones a bins 
def pos_to_bin(p, b=50, overlap=False):

    if overlap:
        p = float(p) - b/2
    else:
        p = float(p)
    bin = floor(p/b)
    #set de nombres de los bins como secuencias sucesivas (1,2,3,4,5 ... N) impares up - pares overlap
    if overlap:
        bin = bin * 2 + 1        
    else:
        bin = bin * 2

    return(bin)

#calcula los RPM de cada bin 
def bin_to_RPM(bin, total_reads):
    return (bin/total_reads)*1000000


# agrupar bins de forma consecutiva y que pasen un trshold determinado 
def bin_group(chromosome, treshold = 0.5):
    consecutive_bins = [[]] #iniciar la generacion de listas
    
    for bin, rpm in enumerate(chromosome): # lista de posiciones (bin) y valores (rpm)
        
        if rpm > treshold: #si el rpm es mayor al treshold
            consecutive_bins[-1].append(bin) # agrega el valor a la lista anterior para generar la susecion de numeros
        elif len(consecutive_bins[-1]) != 0: #si un valor no cumple la condicion y ademas la lista anterior tiene valores dentro genera una nueva lista
            consecutive_bins.append([])

    if len(consecutive_bins[-1]) == 0:
         consecutive_bins.remove([])
    return consecutive_bins


#clusteriza los bins de acuerdo a su distancia con otros clusters secuenciales
def clustering_bins(bins, treshold = 2):

    clusters = [bins.pop(0)] #extrae la primera lista de clusters
    for cluster in bins:

        if abs(clusters[-1][-1] - cluster[0]) <= treshold #revisa si la distancia entre los ultimos y los primeros valores de cada grupo de bins 
            clusters[-1] += cluster 
        else:
            clusters.append(cluster) 

    return clusters

In [3]:
#lectura del archivo bam 
bam_file = pysam.AlignmentFile('SRR769610_trim.bam','rb')
#header - contiene informacion sobre el alineamiento y los cromosomas
header = bam_file.header.to_dict()

#obtencion de reads e informacion por cromosoma
chromosomes = {}

for i in header['SQ']:
    chromosome_name = i['SN']

    #chromosomes[i['SN']] =  #para sacar los reads de un cromosoma se usa fecth
    #i['LN'] es el largo
    reads = bam_file.fetch(i['SN'])
    
     #lista de posiciones de los reads
    pos = []
    for read in reads:
        p = (read.to_dict())['ref_pos']
        pos.append(p)

    #obtencion de bins 
    bins = [pos_to_bin(p) for p in pos]
    o_bins = [pos_to_bin(p, overlap=True) for p in pos]
    c = collections.Counter(bins + o_bins)
    chromosomes[chromosome_name] = dict(c)

In [210]:
total_reads = bam_file.mapped
rpm_per_bin = {}

for k,v in chromosomes.items():
    bin_per_chr = [bin_to_RPM(bin, total_reads) for bin in list(v.values())]
    rpm_per_bin[k] = bin_per_chr
    

In [261]:
test =bin_group(rpm_per_bin['Mt'], treshold = 3)
len(test)

test_clust = clustering_bins(test)

In [262]:
test_clust

[[0, 1, 2, 3, 4],
 [8, 10, 11, 13, 14, 15, 16],
 [24, 26, 27, 28, 29, 30, 31, 32, 33, 35],
 [40, 41, 43, 45, 47, 48, 49, 50, 51, 52, 54],
 [58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72],
 [75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  109,
  110,
  111,
  112,
  113,
  114,
  116,
  117,
  118,
  119],
 [122],
 [125,
  126,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165],
 [170, 171, 172, 173, 174, 175, 176, 178, 179],
 [187,
  188,
  189,
  190,
  192,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  207,
  208,
  209,
  210,
  211],
 [214,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  2

In [256]:
clusters

[[0, 1, 2, 3, 4],
 [8, 10, 11, 13, 14, 15, 16],
 [24, 26, 27, 28, 29, 30, 31, 32, 33, 35],
 [40, 41, 43, 45, 47, 48, 49, 50, 51, 52, 54],
 [58, 59, 61, 62, 64, 66, 67, 68, 70, 71, 72],
 [75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  109,
  110,
  111,
  112,
  113,
  114,
  116,
  117,
  118,
  119],
 [122],
 [125,
  126,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165],
 [170, 171, 172, 173, 174, 175, 176, 178, 179],
 [187,
  188,
  189,
  190,
  192,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  207,
  208,
  209,
  210,
  211],
 [214,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  2

In [165]:
for i in range(len(test)):
    j = i

    if i >= len(test) - 1:
        break
    
    if abs(test[i][-1] - test[j+1][0]) <= 2:
        for num in test[j+1]:
            test[i].append(num)
            if num in test[i]:
                pass
            else:
                break
        #test[i].append(test[j+1])
            print(test[i][-1], test[j+1][0], 'deberian ser una lista', test[j+1], test[i])
   
    else:
        print(test[i][-1], test[j+1][0], 'lista separada')
    

    #if abs(i[-1] - i[0]) < 1

4 8 lista separada
10 10 deberian ser una lista [10, 11] [8, 10]
11 10 deberian ser una lista [10, 11] [8, 10, 11]
13 13 deberian ser una lista [13, 14, 15, 16] [10, 11, 13]
14 13 deberian ser una lista [13, 14, 15, 16] [10, 11, 13, 14]
15 13 deberian ser una lista [13, 14, 15, 16] [10, 11, 13, 14, 15]
16 13 deberian ser una lista [13, 14, 15, 16] [10, 11, 13, 14, 15, 16]
16 24 lista separada
26 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26]
27 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27]
28 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27, 28]
29 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27, 28, 29]
30 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27, 28, 29, 30]
31 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27, 28, 29, 30, 31]
32 26 deberian ser una lista [26, 27, 28, 29, 30, 31, 32, 33] [24, 26, 27, 28, 29, 30, 31, 32]
33 26 deberian ser una 

In [167]:
test[:10]

[[0, 1, 2, 3, 4],
 [8, 10, 11],
 [10, 11, 13, 14, 15, 16],
 [13, 14, 15, 16],
 [24, 26, 27, 28, 29, 30, 31, 32, 33],
 [26, 27, 28, 29, 30, 31, 32, 33, 35],
 [35],
 [40, 41, 43],
 [43, 45],
 [45, 47, 48, 49, 50, 51, 52]]